# Experiments

In [31]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

# For type annotations
import numpy as np

## Glass

In [32]:
dataset_name = 'glass'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [33]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

TODO: Get random `i`

In [34]:
# i = 134 is also a nice value to study
i = 138
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138
1/1 [==============================] - 0s 17ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.0007575714
Network output: (<class 'numpy.int64'>) 1


In [35]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

### Improving the Explanation

In [36]:
import docplex

In [37]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [38]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

In [39]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [40]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,2.967691214515491),
 docplex.mp.LinearConstraint[input GE](x_0,GE,2.967691214515491),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.408120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [41]:
def log_and_improve_explanation(minimal_explanation: list, epsilon: float):
	for constraint in minimal_explanation:
		testing_model.solve()
		print('Initial constraint:' + '\t', constraint)

		variable = constraint.lhs
		while testing_model.solution is None:
			if constraint.sense == docplex.mp.constants.ComparisonType.LE:
				if constraint.rhs.constant <= variable.ub:
					constraint.rhs += epsilon
				else:
					break
			elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
				if constraint.rhs.constant >= variable.lb:
					constraint.rhs -= epsilon
				else:
					break
			else:
				raise Exception('Constraint sense was neither LE nor GE')

			testing_model.solve()

		# Undo last operation
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			constraint.rhs -= epsilon
		elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
			constraint.rhs += epsilon

		print('Final constraint:' + '\t', constraint)
		print()

In [42]:
log_and_improve_explanation(linear_constraints, epsilon=0.01)

Initial constraint:	 input LE: x_0 <= 2.967691214515491
Final constraint:	 input LE: x_0 <= 5.127691214515445

Initial constraint:	 input GE: x_0 >= 2.967691214515491
Final constraint:	 input GE: x_0 >= 1.427691214515511

Initial constraint:	 input LE: x_3 <= -1.408120229258977
Final constraint:	 input LE: x_3 <= -1.408120229258977

Initial constraint:	 input GE: x_3 >= -1.408120229258977
Final constraint:	 input GE: x_3 >= -1.518120229258977

Initial constraint:	 input LE: x_5 <= -0.790702170757714
Final constraint:	 input LE: x_5 <= -0.790702170757714

Initial constraint:	 input GE: x_5 >= -0.790702170757714
Final constraint:	 input GE: x_5 >= -0.790702170757714

Initial constraint:	 input LE: x_6 <= 4.24127975754059
Final constraint:	 input LE: x_6 <= 4.24127975754059

Initial constraint:	 input GE: x_6 >= 4.24127975754059
Final constraint:	 input GE: x_6 >= 4.24127975754059

Initial constraint:	 input LE: x_7 <= -0.3615292659832898
Final constraint:	 input LE: x_7 <= -0.36152926598

In [43]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.427691214515511),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.518120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [44]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [45]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.427691214515511),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.518120229258977),
 docplex.mp.LinearConstraint[input](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input](x_8,EQ,-0.6037614142464092)]

### Pretty Printing

In [46]:
def get_variable_index(variable: docplex.mp.dvar.Var) -> int:
	index = variable.name.split('_')[1]
	return int(index)

In [47]:
def represent_constraint_with_feature_name(constraint: docplex.mp.constr.LinearConstraint):
	variable = constraint.lhs
	index = get_variable_index(variable)
	feature_name = dataframe.columns[index]
	return f'{feature_name} {constraint.sense.operator_symbol} {constraint.rhs}'

In [48]:
def simple_print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		print(represent_constraint_with_feature_name(e))

In [49]:
def group_by_name(constraints: list[docplex.mp.constr.LinearConstraint]):
	group = {}
	for c in constraints:
		variable = c.lhs
		i = get_variable_index(variable)
		feature_name = dataframe.columns[i]
		if feature_name not in group:
			group[feature_name] = []
		group[feature_name].append(c)

	return group

In [50]:
def get_interval_distance(constraints: list[docplex.mp.constr.LinearConstraint]):
	if len(constraints) == 1:
		return 0
	elif len(constraints) == 2:
		c_0 = constraints[0]
		c_1 = constraints[1]
		assert c_0.lhs.name == c_1.lhs.name
		distance = c_0.rhs.constant - c_1.rhs.constant
		return abs(distance)
	else:
		ValueError()

In [51]:
def pretty_print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	group = group_by_name(improved_explanation)
	for (feature, constraints) in group.items():
		if len(constraints) == 2:
			c_0 = constraints[0]
			c_1 = constraints[1]
			le_operator = docplex.mp.constants.ComparisonType.LE
			if c_0.sense == le_operator:
				c_plus = c_0
				c_minus = c_1
			else:
				c_minus = c_0
				c_plus = c_1
			print(f'{c_minus.rhs} {le_operator.operator_symbol} {feature} {le_operator.operator_symbol} {c_plus.rhs}')
		else:
			print(represent_constraint_with_feature_name(constraints[0]))

In [52]:
pretty_print_explanation(improved_explanation)

1.427691214515511 <= RI <= 5.127691214515445
-1.518120229258977 <= Al <= -1.408120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [53]:
group = group_by_name(improved_explanation)
for feature in group:
	print(get_interval_distance(group[feature]))

3.699999999999934
0.1100000000000001
0
0
0
0


In [54]:
group

{'RI': [docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
  docplex.mp.LinearConstraint[input GE](x_0,GE,1.427691214515511)],
 'Al': [docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
  docplex.mp.LinearConstraint[input GE](x_3,GE,-1.518120229258977)],
 'K': [docplex.mp.LinearConstraint[input](x_5,EQ,-0.790702170757714)],
 'Ca': [docplex.mp.LinearConstraint[input](x_6,EQ,4.24127975754059)],
 'Ba': [docplex.mp.LinearConstraint[input](x_7,EQ,-0.3615292659832898)],
 'Fe': [docplex.mp.LinearConstraint[input](x_8,EQ,-0.6037614142464092)]}

### Metric-ify

In [55]:
from math import sqrt

def get_diameter(feature_ranges: list[float]) -> float:
	squared_sum = 0
	for range_ in feature_ranges:
		squared_sum += range_ ** 2
	return sqrt(squared_sum)

In [56]:
ranges = [get_interval_distance(group[feature]) for feature in group_by_name(improved_explanation)]

get_diameter(ranges)

3.7016347739883133

### Anchor

In [57]:
from anchor import utils

In [58]:
d = utils.load_csv_dataset(
	data=f'datasets/{dataset_name}/test.csv',
	target_idx=-1,
	feature_names=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','target'],
	skip_first=True
)

In [59]:
from anchor import anchor_tabular

In [60]:
explainer = anchor_tabular.AnchorTabularExplainer(
	d.class_names,
	d.feature_names,
	d.train,
	d.categorical_names)

In [61]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [62]:
exp = explainer.explain_instance(data[i, :-1], predict_fn)

1/1 [==============================] - 0s 17ms/step


In [63]:
for name in exp.names():
	print(name)

RI > -0.64
K > -0.60
Na <= 0.84
Si > -0.81
Al > -0.57


In [64]:
pretty_print_explanation(improved_explanation)

1.427691214515511 <= RI <= 5.127691214515445
-1.518120229258977 <= Al <= -1.408120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [65]:
simple_print_explanation(improved_explanation)

RI <= 5.127691214515445
RI >= 1.427691214515511
Al <= -1.408120229258977
Al >= -1.518120229258977
K == -0.790702170757714
Ca == 4.24127975754059
Ba == -0.3615292659832898
Fe == -0.6037614142464092


In [66]:
print('Precision: %f' % exp.precision())
print('Coverage:  %f' % exp.coverage())

Precision: 0.252684
Coverage:  0.386700


In [67]:
ranges = [get_interval_distance(group[feature]) for feature in group_by_name(improved_explanation)]

get_diameter(ranges)

3.7016347739883133

## Australian

In [68]:
dataset_name = 'australian'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [69]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

TODO: Get random `i`

In [70]:
i = 138
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138
1/1 [==============================] - 0s 21ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.03920661
Network output: (<class 'numpy.int64'>) 1


In [71]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input5](x_4,EQ,1.528974678604176),
 docplex.mp.LinearConstraint[input6](x_5,EQ,4.0),
 docplex.mp.LinearConstraint[input8](x_7,EQ,1.0),
 docplex.mp.LinearConstraint[input11](x_10,EQ,0),
 docplex.mp.LinearConstraint[input12](x_11,EQ,2.0),
 docplex.mp.LinearConstraint[input13](x_12,EQ,-1.0696373068996796)]

### Improving the Explanation

In [72]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [73]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input5](x_4,EQ,1.528974678604176),
 docplex.mp.LinearConstraint[input6](x_5,EQ,4.0),
 docplex.mp.LinearConstraint[input8](x_7,EQ,1.0),
 docplex.mp.LinearConstraint[input11](x_10,EQ,0),
 docplex.mp.LinearConstraint[input12](x_11,EQ,2.0),
 docplex.mp.LinearConstraint[input13](x_12,EQ,-1.0696373068996796)]

In [74]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [75]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_4,LE,1.528974678604176),
 docplex.mp.LinearConstraint[input GE](x_4,GE,1.528974678604176),
 docplex.mp.LinearConstraint[input LE](x_5,LE,4.0),
 docplex.mp.LinearConstraint[input GE](x_5,GE,4.0),
 docplex.mp.LinearConstraint[input LE](x_7,LE,1.0),
 docplex.mp.LinearConstraint[input GE](x_7,GE,1.0),
 docplex.mp.LinearConstraint[input LE](x_10,LE,0),
 docplex.mp.LinearConstraint[input GE](x_10,GE,0),
 docplex.mp.LinearConstraint[input LE](x_11,LE,2.0),
 docplex.mp.LinearConstraint[input GE](x_11,GE,2.0),
 docplex.mp.LinearConstraint[input LE](x_12,LE,-1.0696373068996796),
 docplex.mp.LinearConstraint[input GE](x_12,GE,-1.0696373068996796)]

In [76]:
log_and_improve_explanation(linear_constraints, epsilon=0.01)

Initial constraint:	 input LE: x_4 <= 1.528974678604176
Final constraint:	 input LE: x_4 <= 1.7989746786041763

Initial constraint:	 input GE: x_4 >= 1.528974678604176
Final constraint:	 input GE: x_4 >= 0.8289746786041754

Initial constraint:	 input LE: x_5 <= 4.0
Final constraint:	 input LE: x_5 <= 4.989999999999979

Initial constraint:	 input GE: x_5 >= 4.0
Final constraint:	 input GE: x_5 >= 2.0100000000000424

Initial constraint:	 input LE: x_7 <= 1.0
Final constraint:	 input LE: x_7 <= 1.0

Initial constraint:	 input GE: x_7 >= 1.0
Final constraint:	 input GE: x_7 >= 0.009999999999999247

Initial constraint:	 input LE: x_10 <= 0
Final constraint:	 input LE: x_10 <= 0.9900000000000007

Initial constraint:	 input GE: x_10 >= 0.9900000000000007
Final constraint:	 input GE: x_10 >= 0.009999999999999913

Initial constraint:	 input LE: x_11 <= 2.0
Final constraint:	 input LE: x_11 <= 2.9999999999999787

Initial constraint:	 input GE: x_11 >= 2.0
Final constraint:	 input GE: x_11 >= 1.0

In [77]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_4,LE,1.7989746786041763),
 docplex.mp.LinearConstraint[input GE](x_4,GE,0.8289746786041754),
 docplex.mp.LinearConstraint[input LE](x_5,LE,4.989999999999979),
 docplex.mp.LinearConstraint[input GE](x_5,GE,2.0100000000000424),
 docplex.mp.LinearConstraint[input LE](x_7,LE,1.0),
 docplex.mp.LinearConstraint[input GE](x_7,GE,0.009999999999999247),
 docplex.mp.LinearConstraint[input LE](x_10,LE,0.009999999999999913),
 docplex.mp.LinearConstraint[input GE](x_10,GE,0.009999999999999913),
 docplex.mp.LinearConstraint[input LE](x_11,LE,2.9999999999999787),
 docplex.mp.LinearConstraint[input GE](x_11,GE,1.0099999999999991),
 docplex.mp.LinearConstraint[input LE](x_12,LE,10.55036269310014),
 docplex.mp.LinearConstraint[input GE](x_12,GE,-1.0696373068996796)]

In [78]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [79]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_4,LE,1.7989746786041763),
 docplex.mp.LinearConstraint[input GE](x_4,GE,0.8289746786041754),
 docplex.mp.LinearConstraint[input LE](x_5,LE,4.989999999999979),
 docplex.mp.LinearConstraint[input GE](x_5,GE,2.0100000000000424),
 docplex.mp.LinearConstraint[input LE](x_7,LE,1.0),
 docplex.mp.LinearConstraint[input GE](x_7,GE,0.009999999999999247),
 docplex.mp.LinearConstraint[input LE](x_11,LE,2.9999999999999787),
 docplex.mp.LinearConstraint[input GE](x_11,GE,1.0099999999999991),
 docplex.mp.LinearConstraint[input LE](x_12,LE,10.55036269310014),
 docplex.mp.LinearConstraint[input GE](x_12,GE,-1.0696373068996796),
 docplex.mp.LinearConstraint[input](x_10,EQ,0.009999999999999913)]

### Pretty Printing

In [80]:
pretty_print_explanation(improved_explanation)

0.8289746786041754 <= A5 <= 1.7989746786041763
2.0100000000000424 <= A6 <= 4.989999999999979
0.009999999999999247 <= A8 <= 1.0
1.0099999999999991 <= A12 <= 2.9999999999999787
-1.0696373068996796 <= A13 <= 10.55036269310014
A11 == 0.009999999999999913


In [81]:
group = group_by_name(improved_explanation)
for feature in group:
	print(get_interval_distance(group[feature]))

0.9700000000000009
2.9799999999999365
0.9900000000000008
1.9899999999999796
11.61999999999982
0


### Anchor

In [82]:
feature_names = dataframe.columns.to_list()

In [83]:
d = utils.load_csv_dataset(
	data=f'datasets/{dataset_name}/test.csv',
	target_idx=-1,
	feature_names=feature_names,
	skip_first=True
)

In [84]:
explainer = anchor_tabular.AnchorTabularExplainer(
	d.class_names,
	d.feature_names,
	d.train,
	d.categorical_names)

In [85]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [86]:
for a in d.train:
	a == data[i]

/tmp/ipykernel_13424/1890216175.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  a == data[i]


In [87]:
exp = explainer.explain_instance(data[i, :-1], predict_fn)

1/1 [==============================] - 0s 17ms/step


In [88]:
for name in exp.names():
	print(name)

In [89]:
pretty_print_explanation(improved_explanation)

0.8289746786041754 <= A5 <= 1.7989746786041763
2.0100000000000424 <= A6 <= 4.989999999999979
0.009999999999999247 <= A8 <= 1.0
1.0099999999999991 <= A12 <= 2.9999999999999787
-1.0696373068996796 <= A13 <= 10.55036269310014
A11 == 0.009999999999999913


In [90]:
simple_print_explanation(improved_explanation)

A5 <= 1.7989746786041763
A5 >= 0.8289746786041754
A6 <= 4.989999999999979
A6 >= 2.0100000000000424
A8 <= 1.0
A8 >= 0.009999999999999247
A12 <= 2.9999999999999787
A12 >= 1.0099999999999991
A13 <= 10.55036269310014
A13 >= -1.0696373068996796
A11 == 0.009999999999999913


In [91]:
print('Precision: %f' % exp.precision())
print('Coverage:  %f' % exp.coverage())

Precision: 1.000000
Coverage:  1.000000


In [92]:
ranges = [get_interval_distance(group[feature]) for feature in group_by_name(improved_explanation)]

get_diameter(ranges)

12.238704996853032

## Auto

In [93]:
dataset_name = 'auto'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [94]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

TODO: Get random `i`

In [95]:
dataframe.sample(n=1).to_numpy()

array([[ 0.69257235, -1.13712399,  1.        ,  0.        ,  1.        ,
         3.        ,  1.        ,  0.        , -0.36158536,  0.12798781,
        -0.32029258,  0.16903131, -0.47139439,  3.        ,  2.        ,
        -0.40054709,  0.        , -0.69390847,  1.27515512, -0.29145957,
         1.08118501,  1.3921319 ,  0.26316796,  0.31627024,  1.23921209,
         1.        ]])

In [96]:
# i = 138
print('i =', i)
# network_input = data[i, :-1]
network_input = dataframe.drop(['target'], axis=1).sample(n=1).to_numpy()
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 13
1/1 [==============================] - 0s 16ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.008684409
Network output: (<class 'numpy.int64'>) 1


In [97]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input4](x_3,EQ,0),
 docplex.mp.LinearConstraint[input5](x_4,EQ,1.0),
 docplex.mp.LinearConstraint[input6](x_5,EQ,3.0),
 docplex.mp.LinearConstraint[input8](x_7,EQ,0),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.2451733158757535),
 docplex.mp.LinearConstraint[input10](x_9,EQ,-0.1492182038657561),
 docplex.mp.LinearConstraint[input11](x_10,EQ,-0.2272672736211136),
 docplex.mp.LinearConstraint[input12](x_11,EQ,-0.4875903213781468),
 docplex.mp.LinearConstraint[input13](x_12,EQ,-0.4022693455437625),
 docplex.mp.LinearConstraint[input14](x_13,EQ,4.0),
 docplex.mp.LinearConstraint[input15](x_14,EQ,2.0),
 docplex.mp.LinearConstraint[input17](x_16,EQ,5.0),
 docplex.mp.LinearConstraint[input18](x_17,EQ,1.1594114349257307),
 docplex.mp.LinearConstraint[input19](x_18,EQ,-1.8619816112087093),
 docplex.mp.LinearConstraint[input20](x_19,EQ,-0.2914595708730563),
 docplex.mp.LinearConstraint[input21](x_20,EQ,1.297475552030508),
 docplex.mp.LinearConstraint[input22](x_21

### Improving the Explanation

In [98]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [99]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input4](x_3,EQ,0),
 docplex.mp.LinearConstraint[input5](x_4,EQ,1.0),
 docplex.mp.LinearConstraint[input6](x_5,EQ,3.0),
 docplex.mp.LinearConstraint[input8](x_7,EQ,0),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.2451733158757535),
 docplex.mp.LinearConstraint[input10](x_9,EQ,-0.1492182038657561),
 docplex.mp.LinearConstraint[input11](x_10,EQ,-0.2272672736211136),
 docplex.mp.LinearConstraint[input12](x_11,EQ,-0.4875903213781468),
 docplex.mp.LinearConstraint[input13](x_12,EQ,-0.4022693455437625),
 docplex.mp.LinearConstraint[input14](x_13,EQ,4.0),
 docplex.mp.LinearConstraint[input15](x_14,EQ,2.0),
 docplex.mp.LinearConstraint[input17](x_16,EQ,5.0),
 docplex.mp.LinearConstraint[input18](x_17,EQ,1.1594114349257307),
 docplex.mp.LinearConstraint[input19](x_18,EQ,-1.8619816112087093),
 docplex.mp.LinearConstraint[input20](x_19,EQ,-0.2914595708730563),
 docplex.mp.LinearConstraint[input21](x_20,EQ,1.297475552030508),
 docplex.mp.LinearConstraint[input22](x_21

In [100]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [101]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_3,LE,0),
 docplex.mp.LinearConstraint[input GE](x_3,GE,0),
 docplex.mp.LinearConstraint[input LE](x_4,LE,1.0),
 docplex.mp.LinearConstraint[input GE](x_4,GE,1.0),
 docplex.mp.LinearConstraint[input LE](x_5,LE,3.0),
 docplex.mp.LinearConstraint[input GE](x_5,GE,3.0),
 docplex.mp.LinearConstraint[input LE](x_7,LE,0),
 docplex.mp.LinearConstraint[input GE](x_7,GE,0),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.2451733158757535),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.2451733158757535),
 docplex.mp.LinearConstraint[input LE](x_9,LE,-0.1492182038657561),
 docplex.mp.LinearConstraint[input GE](x_9,GE,-0.1492182038657561),
 docplex.mp.LinearConstraint[input LE](x_10,LE,-0.2272672736211136),
 docplex.mp.LinearConstraint[input GE](x_10,GE,-0.2272672736211136),
 docplex.mp.LinearConstraint[input LE](x_11,LE,-0.4875903213781468),
 docplex.mp.LinearConstraint[input GE](x_11,GE,-0.4875903213781468),
 docplex.mp.LinearConstraint[input LE](x_12,

In [102]:
log_and_improve_explanation(linear_constraints, epsilon=0.01)

Initial constraint:	 input LE: x_3 <= 0
Final constraint:	 input LE: x_3 <= 0.9900000000000007

Initial constraint:	 input GE: x_3 >= 0.9900000000000007
Final constraint:	 input GE: x_3 >= 0.009999999999999913

Initial constraint:	 input LE: x_4 <= 1.0
Final constraint:	 input LE: x_4 <= 1.9900000000000009

Initial constraint:	 input GE: x_4 >= 1.0
Final constraint:	 input GE: x_4 >= 0.009999999999999247

Initial constraint:	 input LE: x_5 <= 3.0
Final constraint:	 input LE: x_5 <= 3.9999999999999787

Initial constraint:	 input GE: x_5 >= 3.0
Final constraint:	 input GE: x_5 >= 1.9675233664528946e-14

Initial constraint:	 input LE: x_7 <= 0
Final constraint:	 input LE: x_7 <= 0.9900000000000007

Initial constraint:	 input GE: x_7 >= 0.9900000000000007
Final constraint:	 input GE: x_7 >= 0.009999999999999913

Initial constraint:	 input LE: x_8 <= -0.2451733158757535
Final constraint:	 input LE: x_8 <= 3.694826684124212

Initial constraint:	 input GE: x_8 >= -0.2451733158757535
Final con

In [103]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_3,LE,0.009999999999999913),
 docplex.mp.LinearConstraint[input GE](x_3,GE,0.009999999999999913),
 docplex.mp.LinearConstraint[input LE](x_4,LE,1.9900000000000009),
 docplex.mp.LinearConstraint[input GE](x_4,GE,0.009999999999999247),
 docplex.mp.LinearConstraint[input LE](x_5,LE,3.9999999999999787),
 docplex.mp.LinearConstraint[input GE](x_5,GE,1.9675233664528946e-14),
 docplex.mp.LinearConstraint[input LE](x_7,LE,0.009999999999999913),
 docplex.mp.LinearConstraint[input GE](x_7,GE,0.009999999999999913),
 docplex.mp.LinearConstraint[input LE](x_8,LE,3.694826684124212),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-2.005173315875755),
 docplex.mp.LinearConstraint[input LE](x_9,LE,2.7907817961342283),
 docplex.mp.LinearConstraint[input GE](x_9,GE,-2.6592182038657435),
 docplex.mp.LinearConstraint[input LE](x_10,LE,2.9727327263788674),
 docplex.mp.LinearConstraint[input GE](x_10,GE,-2.597267273621102),
 docplex.mp.LinearConstraint[input LE](x_11,LE

In [104]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [105]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_4,LE,1.9900000000000009),
 docplex.mp.LinearConstraint[input GE](x_4,GE,0.009999999999999247),
 docplex.mp.LinearConstraint[input LE](x_5,LE,3.9999999999999787),
 docplex.mp.LinearConstraint[input GE](x_5,GE,1.9675233664528946e-14),
 docplex.mp.LinearConstraint[input LE](x_8,LE,3.694826684124212),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-2.005173315875755),
 docplex.mp.LinearConstraint[input LE](x_9,LE,2.7907817961342283),
 docplex.mp.LinearConstraint[input GE](x_9,GE,-2.6592182038657435),
 docplex.mp.LinearConstraint[input LE](x_10,LE,2.9727327263788674),
 docplex.mp.LinearConstraint[input GE](x_10,GE,-2.597267273621102),
 docplex.mp.LinearConstraint[input LE](x_11,LE,2.502409678621844),
 docplex.mp.LinearConstraint[input GE](x_11,GE,-2.4075903213781396),
 docplex.mp.LinearConstraint[input LE](x_12,LE,2.90773065445622),
 docplex.mp.LinearConstraint[input GE](x_12,GE,-2.0322693455437633),
 docplex.mp.LinearConstraint[input LE](x_13,LE,5.99

### Pretty Printing

In [106]:
pretty_print_explanation(improved_explanation)

0.009999999999999247 <= num-of-doors <= 1.9900000000000009
1.9675233664528946e-14 <= body-style <= 3.9999999999999787
-2.005173315875755 <= wheel-base <= 3.694826684124212
-2.6592182038657435 <= length <= 2.7907817961342283
-2.597267273621102 <= width <= 2.9727327263788674
-2.4075903213781396 <= height <= 2.502409678621844
-2.0322693455437633 <= curb-weight <= 2.90773065445622
4.099151573733195e-14 <= engine-type <= 5.999999999999957
0.00999999999999836 <= num-of-cylinders <= 5.999999999999915
6.230779781013496e-14 <= fuel-system <= 6.999999999999957
-1.7605885650742714 <= bore <= 1.9394114349257314
-2.191981611208705 <= stroke <= 1.8280183887912935
-0.7914595708730567 <= compression-ratio <= 3.2185404291269193
-1.7225244479694943 <= horsepower <= 1.457475552030508
-2.160184437550108 <= peak-rpm <= 2.1398155624498885
-1.8695904536918801 <= city-mpg <= 3.620409546308088
-2.1489265502805375 <= highway-mpg <= 3.3610734497194334
-1.777543401409989 <= price <= 1.6224565985900137
aspiration 

In [107]:
group = group_by_name(improved_explanation)
for feature in group:
	print(get_interval_distance(group[feature]))

1.9800000000000015
3.999999999999959
5.699999999999967
5.449999999999972
5.56999999999997
4.909999999999984
4.9399999999999835
5.9999999999999165
5.989999999999917
6.999999999999895
3.700000000000003
4.019999999999999
4.009999999999976
3.1800000000000024
4.299999999999996
5.489999999999968
5.509999999999971
3.4000000000000026
0
0


### Anchor

In [108]:
feature_names = dataframe.columns.to_list()

In [109]:
print('Precision: %f' % exp.precision())
print('Coverage:  %f' % exp.coverage())

Precision: 1.000000
Coverage:  1.000000


In [110]:
ranges = [get_interval_distance(group[feature]) for feature in group_by_name(improved_explanation)]

get_diameter(ranges)

20.715276971356044